In [4]:
pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 14.5 MB/s eta 0:00:00


In [5]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.9 MB/s eta 0:00:00


In [1]:
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report
import random
import itertools
import numpy as np
import scipy
from sklearn import metrics
from sklearn_crfsuite.utils import flatten
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [3]:
dataset = load_dataset("conll2003")
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# A function to convert digit labels to string labels
def convert_ner_tags(dataset):
  # Label mapping from integer to string
  label_map = {0: 'O', 1: 'PER', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}
  list_sent = []
  list_labels = []
  for sent in dataset:
    words = []
    label = []
    for word, label_idx in zip(sent['tokens'], sent['ner_tags']):
        label.append(label_map[label_idx])
        words.append(word)
    list_sent.append(words)
    list_labels.append(label)
  return list_sent, list_labels

In [5]:
# Define the train, validation and test set
train_ner, train_label_ner = convert_ner_tags(train_set)
validation_ner, validation_label_ner  = convert_ner_tags(validation_set)
test_ner, test_label_ner = convert_ner_tags(test_set)

In [6]:
print('Train',len(train_ner),len(train_label_ner))
print('Test',len(test_ner),len(test_label_ner))
print('Validation',len(validation_ner),len(validation_label_ner))

Train 14041 14041
Test 3453 3453
Validation 3250 3250


In [7]:
# Increase size of train set
new_train = []
new_train.extend(train_ner)
new_train.extend(validation_ner)

new_label = []
new_label.extend(train_label_ner)
new_label.extend(validation_label_ner)

print('Full',len(new_train),len(new_label))

Full 17291 17291


### Train CRF Model

In [43]:
# Initialize the CRF model with class weighting
crf = CRF(algorithm='lbfgs', c1=0.5, c2=2, max_iterations=400, all_possible_states=True)
crf.fit(new_train, new_label)
labels = list(crf.classes_)

### Evaluation

In [44]:
y_pred = crf.predict(test_ner)
print(test_label_ner[0])
print(y_pred[0])

['O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'PER', 'O', 'O', 'O', 'O']
['I-PER', 'O', 'I-ORG', 'O', 'I-PER', 'B-ORG', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O']


In [45]:
# Calculate evaluation metrics
report = classification_report(flatten(test_label_ner), flatten(y_pred), digits=3)
print(report)

              precision    recall  f1-score   support

       B-LOC      0.324     0.128     0.184       257
      B-MISC      0.171     0.176     0.174       216
       B-ORG      0.266     0.177     0.213       835
       B-PER      0.519     0.530     0.525      1156
       I-LOC      0.241     0.070     0.108       702
       I-ORG      0.403     0.329     0.362      1668
       I-PER      0.391     0.184     0.251      1661
           O      0.907     0.966     0.936     38323
         PER      0.509     0.382     0.437      1617

    accuracy                          0.848     46435
   macro avg      0.415     0.327     0.354     46435
weighted avg      0.819     0.848     0.830     46435



In [53]:
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV

In [54]:
labels = list(filter(lambda a: a != 'O', labels))
print(labels)

['I-PER', 'I-LOC', 'PER', 'B-PER', 'I-ORG', 'B-ORG', 'B-MISC', 'B-LOC']


In [58]:
%%time
crf3 = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring='f1_macro')

rs.fit(new_train, new_label)

AttributeError: ignored